<a href="https://colab.research.google.com/github/potterton48/Notebooks/blob/main/TPD_Workshop1_ML_Training_Set_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop 1: Find if a PDB is seen in the training set of RosettaFoldTTA All-Atom (or any model trained on the PDB)
Steps of the protocol:
1. Take PDBs from previous exercise (to begin with you may want to run through the example
2. Check if PDB ID is in training list
3. Download fasta file of sequence from PDB
4. Search sequence similarity of query sequence to the training set
5. (Extension) Try to find structure similarity of protein to training set.

**We need to install some packages, this may take around 1-2 minutes to complete.**

In [ ]:
# Installation
!apt install hmmer  # like blast for fast sequence database searching
!pip install pandas
!pip install gdown  # for downloading a file from google drive
!pip install biopython  # used for handling sequences and pdb files
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install bioconda::mafft # to do MSA

In [ ]:
# Imports
import requests
import os
import re
import subprocess
import tempfile
from pathlib import Path

import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

**Downloading sequence file of RosettaFold training set**

This file was created by checking the date of the PDB file, RosettaFold was trained on a 100 non-redundant set of the PDB from March 2021.

In [ ]:
# Downloading sequence file of RosettaFold training set
!gdown https://drive.google.com/uc?id=1HdsA87s0hDtvMF2xc8BVMOM0wIJocqKN

**Check if PDB ID is in training list**

If this function returns True that means that the exact PDB was seen in the training set of RoseTTAFold-AllAtom. This means that prediction of the structure will be essentially memorisation.

In [ ]:
def check_pdb_in_training_set(pdb_id: str) -> bool:
    """This function checks whether PDB is seen in the training set of RoseTTAFold-AllAtom.
    Args:
        pdb_id: 4 letter accession code e.g. 3PWH
    Returns:
        bool, True if pdb seen in training set.
    """
    for record in SeqIO.parse('/content/rosettafold_pdb_seqres.txt', "fasta"):
        if record.id[1:].upper().startswith(pdb_id.upper()):
            return True
    return False

check_pdb_in_training_set('7LPS')

**Download fasta file for PDB of interest**

In [ ]:
def download_pdb_fasta(pdb_id: str) -> None | str:
    """ Downloads the FASTA sequence for a given PDB ID and saves it to a file.
    Args:
    - pdb_id (str): The PDB ID of the protein (4 characters).
    Returns:
        string of filename (e.g. 1A2B.fasta)
    """
    # Convert the PDB ID to uppercase
    pdb_id = pdb_id.upper()
    # RCSB PDB URL to fetch the FASTA sequence for the given PDB ID
    url = f"https://www.rcsb.org/fasta/entry/{pdb_id}"
    # Send a GET request to the PDB website to fetch the FASTA sequence
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Save the FASTA content to the specified output file
        output_file = f'{pdb_id}.fasta'
        with open(output_file, 'w') as fasta_file:
            fasta_file.write(response.text)
        print(f"FASTA sequence for PDB ID {pdb_id} saved to {output_file}.")
        return output_file
    print(f"Failed to retrieve FASTA for PDB ID {pdb_id}. HTTP Status Code: {response.status_code}")
    return None

# Example usage:
fasta_file = download_pdb_fasta("7LPS")

**Split fasta file by chain**

In [ ]:
def split_fasta_by_chains(fasta_file: str) -> list[str]:
    """ This function takes the fasta file from the PDB and split into separate fasta files per chain
    Args:
        fasta_file: Filename and path of input to be split
    Returns:
        List of fasta file paths for each chain.
    """
    with open(fasta_file, 'r') as file:
        lines = file.readlines()
    chain_sequences = {}
    for line in lines:
        if line.startswith(">"):  # Header line
            header = line.strip()
            chains = [c.strip() for c in header.split('|')[1].replace("Chains", "").split(',')]
            for chain in chains:
                chain_sequences[chain] = [header]
        else:
            for chain in chains:
                chain_sequences[chain].append(line.strip())
    output_files = []
    for chain, seq_lines in chain_sequences.items():
        fname = f"{Path(fasta_file).stem}_{chain}.fasta"
        output_files.append(fname)
        sequence = ''.join(seq_lines[1:])  # Combine sequence lines
        split_sequence = [sequence[i:i+60] for i in range(0, len(sequence), 60)]  # Split into 60-char lines
        with open(fname, 'w') as out_file:
            out_file.write(f"{seq_lines[0]}\n")  # Write header
            out_file.write('\n'.join(split_sequence) + '\n')  # Write sequence in 80-char lines
    return output_files

chain_fasta_files = split_fasta_by_chains(fasta_file)

**These are the PDB chain fasta files, we can only run one at once**

In [ ]:
print(chain_fasta_files)

In [ ]:
# Pick which chain you want from list above. Remember python is zero indexed (i.e. counting starts from zero)
query_fasta_file = chain_fasta_files[4]

**Now we need to check if query sequence is anything like any of the sequences in RoseTTAFoldAllAtom training set. We use phmmer a tool akin to blastp to quickly find results.**

In [ ]:
def process_phmmer_results(input_file: str) -> pd.DataFrame:
    """ Takes results from phmmer and returns a dataframe
    Args:
        input_file: output tblout from phmmer
    Returns:
        DataFrame with 2 columns: PDB_ID, e_value
    """
    with open(input_file) as f:
        readlines = f.readlines()
    readlines = [line for line in readlines if not line.startswith('#')]
    e_values: list[float] = []
    pdb_ids: list[str] = []
    for line in readlines:
        split_line = line.split()
        e_values.append(float(split_line[4]))
        pdb_ids.append(split_line[0][1:])
    df = pd.DataFrame()
    df['PDB_ID'] = pdb_ids
    df['e_value'] = e_values
    return df


def run_phmmer(query_fasta_file: str, seq_db: str, output_dir: str = tempfile.mkdtemp()) -> pd.DataFrame:
    """ Runs phmmer on a query fasta file returns dataframe of results
    Args:
        query_fasta_file: input fasta file to use as query search
        output_dir: where to save data, defaults to temp folder
        seq_db: sequence file to screen against
    Returns:
        DataFrame with 2 columns: PDB_ID, e_value
    """
    output_file = os.path.join(
        output_dir, f'{Path(query_fasta_file).stem}_{Path(seq_db).stem}.tab',
    )
    run_file = os.path.join(output_dir, 'run_file.txt')
    subprocess.run([
        'phmmer', '--noali', '--tblout', output_file,
        '-o', run_file, query_fasta_file, seq_db,
    ])
    df = process_phmmer_results(output_file)
    os.remove(run_file)
    os.remove(output_file)
    return df

df = run_phmmer(query_fasta_file=query_fasta_file, seq_db = '/content/rosettafold_pdb_seqres.txt', output_dir='/content/')

**Once we've found similar proteins in the training set, we will want to grab their sequences so we can do a MSA.**

In [ ]:
def find_off_target_seqs(ids: list[str]) -> list[SeqRecord]:
    """Finds off target sequences from their id
    Args:
        Input list of ids from phmmer.
    Returns:
        List of Biopython SeqRecord for each off target record found
    """
    record_list = []
    for record in SeqIO.parse('/content/rosettafold_pdb_seqres.txt', "fasta"):
        if record.id[1:] in match_ids:
            record_list.append(record)
        if len(record_list) == len(match_ids):
            break
    return record_list

match_ids = df.PDB_ID.to_list()
record_list = find_off_target_seqs(match_ids)

**Running an MSA to get sequence identity of each off-target**

In [ ]:
# Do quick MSA of results from DF against query sequence and return sequence identity
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO

def calculate_identity(seq1: str, seq2: str) -> float:
    """ Calculate the percentage of identical positions between two aligned sequences.
    Args:
        seq1 (str): First sequence (query sequence).
        seq2 (str): Second sequence (aligned sequence).
    Returns:
        identity (float): Percentage of identical positions.
    """
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b and a != '-')
    return matches / len(seq1) * 100


def run_msa_and_calc_identity(query_fasta: str, sequence_records: list[SeqRecord], msa_output: str = "aligned.fasta") -> dict[str, float]:
    """ Perform MSA on a list of SeqRecord sequences with a query sequence and calculate sequence identity.
    Args:
        query_fasta (str): Path to the query FASTA file.
        sequence_records (list): List of SeqRecord objects to be aligned with the query.
        msa_output (str): Path to save the aligned MSA output.
    Returns:
        identity_dict (dict): Dictionary with SeqRecord ids and their identity percentages compared to the query.
    """
    # Write the query and sequences to a temporary FASTA file
    with open("temp_sequences.fasta", "w") as temp_fasta:
        # Write the query sequence
        for record in SeqIO.parse(query_fasta, "fasta"):
            SeqIO.write(record, temp_fasta, "fasta")
            query_id = record.id  # Save the query ID for reference
        # Write each sequence from sequence_records to the temp file
        for record in sequence_records:
            SeqIO.write(record, temp_fasta, "fasta")
    # Run MAFFT to perform MSA
    mafft_cline = MafftCommandline(input="temp_sequences.fasta")
    mafft_cline.set_parameter("--auto", True)  # You can set MAFFT parameters here
    stdout, stderr = mafft_cline()
    # Write MAFFT output to file
    with open(msa_output, "w") as aligned_file:
        aligned_file.write(stdout)
    # Read the alignment output
    alignment = AlignIO.read(msa_output, "fasta")
    # Calculate sequence identity
    query_seq = None
    identity_dict = {}
    # Find the query sequence in the alignment
    for record in alignment:
        if record.id == query_id:
            query_seq = str(record.seq)
            break
    # Compare query sequence with each aligned sequence and calculate sequence identity
    for record in alignment:
        if record.id != query_id:
            aligned_seq = str(record.seq)
            identity = calculate_identity(query_seq, aligned_seq)
            identity_dict[record.id[1:]] = round(identity, 3)  # Use SeqRecord id as key
    # Clean up temporary file
    os.remove("temp_sequences.fasta")
    return identity_dict

seq_dict = run_msa_and_calc_identity(query_fasta_file, record_list)
df['Seq_ID'] = df.PDB_ID.map(seq_dict)

**The results shown as a table**

3 columns are found in the table:
*   PDB_ID is the PDB ID + Chain of the similar protein found in the training set
*   e_value is the signficance value of the phmmer search.
*   Seq_ID is the sequence identity of the query vs the protein found in the training set expressed as a %. Above 70% is very high sequence similarity. Above 30-40% indicate it is likely to have the same overall fold or be in the same superfamily.

In [ ]:
df